# Unsecured System

# Lab 1: Creating an Unsecured System

## Introduction

Welcome to the first lab of our Zero Trust course! In this lab, we'll set up a basic, unsecured system in AWS using AWS CLI commands and Python (boto3). This hands-on approach will help you understand each component of the system and why better security practices are necessary.

## Objectives

By the end of this lab, you will:
1. Create a Virtual Private Cloud (VPC) with a public subnet
2. Set up an Internet Gateway and configure routing
3. Create a security group with unrestricted access
4. Deploy a "sensitive" backend server
5. Launch a public-facing web server


![image.png](zt-lab1-insecure-network.png)

## Setup

First, let's set up our AWS environment. Make sure you have configured your AWS CLI with the appropriate credentials.


In [ ]:
# Import necessary AWS SDK and utility modules

import boto3
import json
import time

## Set the AWS region

Configure the AWS region to create the network and servers

In [ ]:
# Set the AWS region for resource creation (change as needed)

region = 'us-west-2'

## Initialize AWS clients

Setup the necessary environment variables

In [ ]:
# Initialize EC2 client and resource objects for interacting with AWS

ec2_client = boto3.client('ec2', region_name=region)
ec2_resource = boto3.resource('ec2', region_name=region)

print(f"AWS environment initialized in region: {region}")

## Step 1: Create a VPC and Subnet

Let's start by creating our Virtual Private Cloud (VPC) and a public subnet.

In [ ]:
# Create VPC

vpc_response = ec2_client.create_vpc(CidrBlock='10.0.0.0/16')
vpc_id = vpc_response['Vpc']['VpcId']
ec2_client.create_tags(Resources=[vpc_id], Tags=[{'Key': 'Name', 'Value': 'UnsecuredVPC'}])
print(f"VPC created with ID: {vpc_id}")

### Access the VPC Service VPC Menu in AWS Management Console.
#### It should look like below:
![image.png](zt-lab1-image-vpcs-VPC-Console.jpeg)

In [ ]:
# Create public subnet

subnet_response = ec2_client.create_subnet(VpcId=vpc_id, CidrBlock='10.0.1.0/24')
subnet_id = subnet_response['Subnet']['SubnetId']
ec2_client.create_tags(Resources=[subnet_id], Tags=[{'Key': 'Name', 'Value': 'PublicSubnet'}])
print(f"Public subnet created with ID: {subnet_id}")

### Access the VPC Service Subnets Menu in AWS Management Console.
#### It should look like below:
![image.png](zt-lab1-image-subnets-VPC-Console.jpeg)

## Step 2: Set up Internet Gateway and Routing

Now, let's create an Internet Gateway and set up the routing table.

In [ ]:
# Create Internet Gateway

igw_name = "UnsecuredIGW"
igw_response = ec2_client.create_internet_gateway(
    TagSpecifications=[
        {
            'ResourceType': 'internet-gateway',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': igw_name
                }
            ]
        }
    ]
)
igw_id = igw_response['InternetGateway']['InternetGatewayId']
ec2_client.attach_internet_gateway(InternetGatewayId=igw_id, VpcId=vpc_id)
print(f"Internet Gateway '{igw_name}' created and attached with ID: {igw_id}")

### Access the VPC Service Internet Gateways Menu in AWS Management Console.
#### It should look like below:
![image.png](zt-lab1-image-igws-VPC-Console.jpeg)

In [ ]:
# Create and configure route table

rt_name = "UnsecuredRT"
rt_response = ec2_client.create_route_table(
    VpcId=vpc_id,
    TagSpecifications=[
        {
            'ResourceType': 'route-table',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': rt_name
                }
            ]
        }
    ]
)
rt_id = rt_response['RouteTable']['RouteTableId']

# Create route to Internet Gateway
ec2_client.create_route(RouteTableId=rt_id, DestinationCidrBlock='0.0.0.0/0', GatewayId=igw_id)

# Associate route table with subnet
ec2_client.associate_route_table(RouteTableId=rt_id, SubnetId=subnet_id)

print(f"Route table '{rt_name}' created and configured with ID: {rt_id}")

### Access the VPC Service Route Tables Menu in AWS Management Console.
#### It should look like below:
![image.png](zt-lab1-image-RouteTables-VPC-Console.png)

## Step 3: Create an Unsecured Security Group

Let's create a security group that allows all inbound traffic.

In [ ]:
# Create Security Group allowing all traffic in and out

sg_name = "UnsecuredSG"
sg_response = ec2_client.create_security_group(
    GroupName=sg_name,
    Description='Allow all inbound traffic',
    VpcId=vpc_id,
    TagSpecifications=[
        {
            'ResourceType': 'security-group',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': sg_name
                }
            ]
        }
    ]
)
sg_id = sg_response['GroupId']

# Configure security group rules
ec2_client.authorize_security_group_ingress(
    GroupId=sg_id,
    IpPermissions=[
        {'IpProtocol': '-1', 'FromPort': -1, 'ToPort': -1, 'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
    ]
)

print(f"Unsecured security group '{sg_name}' created with ID: {sg_id}")

### Access the VPC Security Groups Menu in AWS Management Console.
#### It should look like below:
![image.png](zt-lab1-image-SecurityGroups-VPC-Console.jpeg)

## Step 4: Deploy Backend Server

Now, let's deploy our backend server with a mock sensitive database.

In [ ]:
# Function to fetch the latest Amazon Linux 2 AMI ID with error handling and fallback

def get_latest_amazon_linux_2_ami():
    try:
        response = ec2_client.describe_images(
            Owners=['amazon'],
            Filters=[
                {'Name': 'name', 'Values': ['amzn2-ami-hvm-*-x86_64-gp2']},
                {'Name': 'state', 'Values': ['available']}
            ]
        )
        
        if not response['Images']:
            print("No Amazon Linux 2 AMIs found. Falling back to a default AMI ID.")
            return 'ami-0caa0a2e2a99b8b82'  # This is a fallback AMI ID, replace with a known good one for your region
        
        # Sort the images by creation date
        sorted_images = sorted(response['Images'], key=lambda x: x['CreationDate'], reverse=True)
        latest_ami_id = sorted_images[0]['ImageId']
        print(f"Latest Amazon Linux 2 AMI ID: {latest_ami_id}")
        return latest_ami_id
    
    except Exception as e:
        print(f"An error occurred while fetching the latest AMI: {str(e)}")
        print("Falling back to a default AMI ID.")
        return 'ami-0caa0a2e2a99b8b82'  # This is a fallback AMI ID, replace with a known good one for your region

# Get the latest Amazon Linux 2 AMI ID
ami_id = get_latest_amazon_linux_2_ami()
print(f"Using AMI ID: {ami_id}")

In [ ]:
# User data script for backend instance setup: installs and configures Apache with PHP

backend_user_data = '''#!/bin/bash
yum update -y
yum install -y httpd php
systemctl start httpd
systemctl enable httpd
cat <<EOF > /var/www/html/api.php
<?php
\$data = ['sensitive' => 'This is sensitive data from the backend'];
header('Content-Type: application/json');
echo json_encode(\$data);
?>
EOF
'''

In [ ]:
# Create and launch the backend EC2 instance with specified configuration

backend_instance = ec2_resource.create_instances(
    ImageId=ami_id,  # Amazon Linux 2 AMI ID (replace with the latest)
    InstanceType='t3.micro',
    MaxCount=1,
    MinCount=1,
    NetworkInterfaces=[{
        'SubnetId': subnet_id,
        'DeviceIndex': 0,
        'AssociatePublicIpAddress': True,
        'Groups': [sg_id]
    }],
    UserData=backend_user_data,
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'BackendServer'
                },
            ]
        },
    ]
)
backend_id = backend_instance[0].id
print(f"Backend server deployed with ID: {backend_id}")

In [ ]:
# Wait for the instance to be running to get the IP addresses
backend_instance[0].wait_until_running()
backend_instance[0].reload()

# Get private IP
backend_private_ip = backend_instance[0].private_ip_address
print(f"Backend server private IP: {backend_private_ip}")

# Get public IP if it exists
backend_public_ip = backend_instance[0].public_ip_address
if backend_public_ip:
    print(f"Backend server public IP: {backend_public_ip}")
else:
    print("Backend server does not have a public IP address.")

### Access the EC2 Security Groups Menu in AWS Management Console.
#### It should look like below:
![image.png](zt-lab1-image-Instances-EC2-us-west-2.jpeg)

## Step 5: Deploy Web Server

Finally, let's deploy our public-facing web server.

In [ ]:
# User data script for frontend instance setup: installs and configures Apache with PHP

web_user_data = f'''#!/bin/bash
yum update -y
yum install -y httpd php
systemctl start httpd
systemctl enable httpd
echo "<?php
\\$backend_ip = '{backend_private_ip}';
\\$data = file_get_contents('http://' . \\$backend_ip . '/api.php');
echo '<h1>Welcome to our Unsecured Web Server!</h1>';
echo '<h2>Data from backend:</h2>';
echo \\$data;
?>" > /var/www/html/index.php
systemctl restart httpd
'''

In [ ]:
# Create and launch the frontend EC2 instance with specified configuration

web_instance = ec2_resource.create_instances(
    ImageId=ami_id,  # Amazon Linux 2 AMI ID (replace with the latest)
    InstanceType='t3.micro',
    MaxCount=1,
    MinCount=1,
    NetworkInterfaces=[{
        'SubnetId': subnet_id,
        'DeviceIndex': 0,
        'AssociatePublicIpAddress': True,
        'Groups': [sg_id]
    }],
    UserData=web_user_data,
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'FrontendServer'
                },
            ]
        },
    ]
)
web_id = web_instance[0].id
print(f"Web server deployed with ID: {web_id}")

In [ ]:
# Wait for the instance to be running to get the IP addresses
web_instance[0].wait_until_running()
web_instance[0].reload()

# Get private IP
web_private_ip = web_instance[0].private_ip_address
print(f"Web server private IP: {web_private_ip}")

# Get public IP
web_public_ip = web_instance[0].public_ip_address
if web_public_ip:
    print(f"Web server public IP: {web_public_ip}")
else:
    print("Web server does not have a public IP address.")

### Access the EC2 Instances Menu in AWS Management Console.
#### It should look like below:
![image.png](zt-lab1-image-Instances2-EC2-us-west-2.jpeg)

## Testing the Unsecured System

Now that our system is set up, let's test it by accessing the web server.

In [ ]:
# Wait for a bit to ensure the servers are fully initialized

import time
import boto3

def wait_for_ec2_instances(instance_ids, region_name, timeout=300, interval=10):
    """
    Waits for the given EC2 instances to pass system and instance status checks within a timeout period.

    Args:
        instance_ids (list): A list of EC2 instance IDs to check.
        region_name (str): The AWS region where the instances are located.
        timeout (int): The maximum time to wait (in seconds) before giving up.
        interval (int): The time interval (in seconds) between status checks.

    Returns:
        bool: True if all instances pass system and instance checks within the timeout, False otherwise.
    """
    ec2 = boto3.client('ec2', region_name=region_name)
    start_time = time.time()

    while time.time() - start_time < timeout:
        # Check the status of the EC2 instances
        response = ec2.describe_instance_status(InstanceIds=instance_ids)
        
        # Get the instance statuses
        statuses = {instance['InstanceId']: {
                        'instance_state': instance['InstanceState']['Name'],
                        'system_status': instance['SystemStatus']['Status'],
                        'instance_status': instance['InstanceStatus']['Status']
                    }
                    for instance in response['InstanceStatuses']}
        
        # Check if all instances are 'running' and both system and instance checks are 'ok'
        all_ready = all(
            status['instance_state'] == 'running' and 
            status['system_status'] == 'ok' and 
            status['instance_status'] == 'ok'
            for status in statuses.values()
        )
        
        if all_ready:
            print(f"All instances {instance_ids} are fully initialized and passed all checks.")
            return True
        
        print(f"Current statuses: {statuses}")
        print(f"Waiting for EC2 instances {instance_ids} to pass system and instance checks...")
        time.sleep(interval)

    print(f"Timeout: EC2 instances {instance_ids} did not fully initialize within {timeout} seconds.")
    return False

# IDs of front-end and back-end EC2 instances
frontend_instance_id = web_id  # Replace with your front-end instance ID
backend_instance_id = backend_id   # Replace with your back-end instance ID
region = 'us-west-2'  # Replace with your region

# Wait for both front-end and back-end instances to be fully initialized
if wait_for_ec2_instances([frontend_instance_id, backend_instance_id], region_name=region):
    print("Both EC2 instances are fully initialized!")
else:
    print("One or both EC2 instances failed to fully initialize within the timeout.")


In [ ]:
# Access the web server

import requests
print("Accessing the web server:")
response = requests.get(f"http://{web_public_ip}")
print(response.text)

### Access the frontend web server by entering the Public IP in a browser tab.
#### It should look like below:
![image.png](zt-lab1-image-frontend.jpeg)

### Access the backend web server by entering the Public IP in a browser tab with /api.php
#### It should look like below:
![image.png](zt-lab1-image-backend.jpeg)

## Conclusion

In this lab, we've set up an unsecured system in AWS and explored its components. We've seen how easy it is to access sensitive information due to the lack of proper security measures. In the upcoming labs, we'll start implementing Zero Trust principles to secure this system.

## Cleanup

To avoid unnecessary charges, let's clean up our resources:

In [ ]:
import time

# Terminate EC2 instances
ec2_client.terminate_instances(InstanceIds=[backend_id, web_id])
print("EC2 instance termination initiated.")

# Wait for instances to fully terminate
def is_terminated(instance_id):
    response = ec2_client.describe_instances(InstanceIds=[instance_id])
    state = response['Reservations'][0]['Instances'][0]['State']['Name']
    return state == 'terminated'

print("Waiting for instances to fully terminate...")
while not (is_terminated(backend_id) and is_terminated(web_id)):
    time.sleep(10)  # Wait for 10 seconds before checking again
    print("Still waiting...")

print("All EC2 instances have been terminated.")

In [ ]:
# Delete security group
ec2_client.delete_security_group(GroupId=sg_id)
print("Security group deleted.")

In [ ]:
# Delete route table association
rt_associations = ec2_client.describe_route_tables(RouteTableIds=[rt_id])['RouteTables'][0]['Associations']
for assoc in rt_associations:
    if assoc['SubnetId'] == subnet_id:
        ec2_client.disassociate_route_table(AssociationId=assoc['RouteTableAssociationId'])
ec2_client.delete_route_table(RouteTableId=rt_id)
print("Route table deleted.")

In [ ]:
# Detach and delete Internet Gateway
ec2_client.detach_internet_gateway(InternetGatewayId=igw_id, VpcId=vpc_id)
ec2_client.delete_internet_gateway(InternetGatewayId=igw_id)
print("Internet Gateway deleted.")

In [ ]:
# Delete subnet
ec2_client.delete_subnet(SubnetId=subnet_id)
print("Subnet deleted.")

In [ ]:
# Delete VPC
ec2_client.delete_vpc(VpcId=vpc_id)
print("VPC deleted.")

In [ ]:
print("All resources have been cleaned up.")